<span style="font-size: 14pt">MIPT, Applied ML, Autumn 2018</span>

<span style="font-size: 16pt"> HW #4: Approximate RL homework

<span style="color:red; font-size: 14pt;"> Дедлайн 19.11.2018 23:59 </span>

<span style="color:blue; font-size: 12pt">Valentin Malykh </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">val@maly.hk</span>


**Оформление дз**: 
- Выполненное задание требуется отправлять через <a href='https://goo.gl/forms/XPSIbwp7wPxB4SsI3'>форму </a>

- Выполненное дз прикрепляйте в формате файла ``<фамилия>_<группа>_task<номер>.ipynb``, например: ``ivanov_594_task4.ipynb`` 

**Вопросы**:
- Вопросы присылайте в канал в телеграмме ``[Fall 2018]ML Seminars``

--------
- **PS1**: Будьте внимательны при заполнении формы, когда отправляете ДЗ 


<h1 align="center">Практическое задание (100%)</h1>
Описание находится по ссылке: https://gist.github.com/madrugado/1262c3077bf7d8ac8166e4350f0f67e2

In [15]:
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
import time
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras import optimizers

In [17]:
env = gym.make("LunarLander-v2")

[2018-11-16 15:10:28,735] Making new env: LunarLander-v2


In [36]:
def Q(s, a, p):
    return model.predict(np.concatenate((s, np.eye(4)[a], p), axis=0).reshape(1, -1))[0][0]

In [37]:
def Q(s, a):
    return model.predict(np.concatenate((s, np.eye(4)[a]), axis=0).reshape(1, -1))[0][0]

def use_prob(episode):
    return 0.05*(1 - episode/num_episodes)

In [19]:
import h5py

In [57]:
# Загрузить модель, обученную на 10000 эпизодах
model = keras.models.load_model("imodel10ktr.mdh5")

In [79]:
# Загрузить модель, обученную на 50000 эпизодах - дает наилучшие результаты
model = keras.models.load_model("model50k.mdh5")

In [78]:
# Загрузить модель, обученную на 100000 эпизодах и учетом луны
model = keras.models.load_model("i512model100ktr.mdh5")

In [ ]:
# Обучить новую модель

dimX = 8 + 4
num_episodes = 10000

model = Sequential()
model.add(Dense(512, activation='relu', input_dim=dimX))
model.add(Dense(1))
model.compile(loss='mse', optimizer=optimizers.adam(lr=0.001), metrics=['accuracy'])

dataX = np.zeros(shape=(1, dimX))
dataY = np.zeros(shape=(1, 1))

over_ind = 0

for episode in range(num_episodes):
    e_dataX = np.zeros(shape=(1, dimX))
    e_dataY = np.zeros(shape=(1, 1))
    
    s = env.reset()
    total_r = 0
    First = True
    
    while True:
        if episode % 100 == 0:
            env.render()
        
        if np.random.rand(1) < use_prob(episode):
            a=env.action_space.sample()
        else:
            a = np.argmax(np.array([Q(s, 0), Q(s, 1), Q(s, 2), Q(s, 3)]))

        string = np.concatenate((s, np.eye(4)[a]), axis=0)

        new_s, r, done, info = env.step(a)


        if First:
            e_dataX[0] = string
            e_dataY[0] = np.array([r])
            dataX[0] = string
            dataY[0] = np.array([r])
            First = False

        e_dataX = np.vstack((e_dataX, string))
        e_dataY = np.vstack((e_dataY, np.array([r])))

        total_r += r

        s = new_s
        
        if done:
#             if total_r >= 200:
#                 r = 1000
            n = len(e_dataY)
            for i in range(1, n):
                e_dataY[n - i - 1][0] = e_dataY[n - i - 1][0] + 0.98 * e_dataY[n - i][0]
            
            print("Episode №", episode, " last reward = ", r, ", total reward = ", total_r)

            if len(dataX) == 1:
                dataX = e_dataX
                dataY = e_dataY
            else:
                dataX = np.concatenate((dataX, e_dataX), axis=0)
                dataY = np.concatenate((dataY, e_dataY), axis=0)

            if len(dataX) > 60000:
                dataX = np.delete(dataX, range(over_ind, over_ind + len(e_dataX)), axis=0)
                dataY = np.delete(dataY, range(over_ind, over_ind + len(e_dataX)), axis=0)
                over_ind += len(e_dataX)
                if over_ind > 59800:
                    over_ind = 0

            if episode%10 == 0:
                model.fit(dataX, dataY, batch_size=32, epochs=2, verbose=0)
            break
env.close()

In [38]:
def play_n_episodes(n, rend = False):
    wins_cnt = 0
    for episode in range(n):
        s = env.reset()
        total_r = 0
        while True:
#             env.render()
            a = np.argmax(np.array([Q(s, 0), Q(s, 1), Q(s, 2), Q(s, 3)]))
            new_s, r, done, info = env.step(a)
            total_r += r
            s = new_s
            if done:
                if total_r >= 200:
                    print("total reward - " + str(total_r) + ", WIN")
                    wins_cnt += 1
                else:
                    print("total reward - " + str(total_r) + ", LOSE")
                break
    print("Wins " + str(wins_cnt) + " of " + str(n) + " episodes.")
    env.close()

In [81]:
play_n_episodes(10)

total reward - 215.90487014632544, WIN
total reward - 228.99222923197436, WIN
total reward - 235.4016250849651, WIN
total reward - 221.47740988970259, WIN
total reward - 212.72065215700349, WIN
total reward - 218.18788523129857, WIN
total reward - 204.07505536510232, WIN
total reward - 216.23714528085668, WIN
total reward - 226.82981262472754, WIN
total reward - 262.94399702637804, WIN
Wins 10 of 10 episodes.
